## ADES demonstration - OGC API Processes 

Deploy a CWL application package as a WPS processing service and submit an execution request

In [ ]:
import requests
import os
import getpass
import json
import yaml
from time import sleep

## Discover a registered process

In [ ]:
rm_endpoint = 'https://catalog.terradue.com/eoepca-services/search'

This is the identifier from the previous demonstration notebook:

In [ ]:
application_package_id = "e563cc5596b02a189277de6228a9e2af"

Query the resource manager catalog to discover the application package:

In [ ]:
payload = {'uid': application_package_id,
           'format': 'json'}

r = requests.get(rm_endpoint, 
                 params=payload)
                 
print(r.status_code, r.reason)

In [ ]:
response = r.json()

print(json.dumps(response, indent=2))

Here's the Application Package CWL:

In [ ]:
application_package = yaml.safe_load(response['features'][0]['properties']['offering']['content']['#text'])

In [ ]:
print(json.dumps(application_package, indent=2))

## Deploy the process

In [ ]:
endpoint = 'http://proc-ades.test.192.168.49.2.nip.io' 

In [ ]:
# token = getpass.getpass('Token:')
token = "eyJhbGciOiJIUzI1NiIsImtpZCI6IlJTQTEifQ.eyJhY3RpdmUiOnRydWUsImV4cCI6MTU5MzUxNTU2NSwiaWF0IjoxNTkzNTExOTY1LCJuYmYiOm51bGwsInBlcm1pc3Npb25zIjpbeyJyZXNvdXJjZV9pZCI6ImI3Y2FkZTVjLTM3MmYtNGM4Ny1iZTgyLWE3OTU2NDk4ZTcyOSIsInJlc291cmNlX3Njb3BlcyI6WyJBdXRoZW50aWNhdGVkIiwib3BlbmlkIl0sImV4cCI6MTU5MzUxNTU2NCwicGFyYW1zIjpudWxsfV0sImNsaWVudF9pZCI6IjYxY2UyOGQ1LWFhMTYtNGRkYy04NDJmLWZjYzE1OGQzMTVmYSIsInN1YiI6bnVsbCwiYXVkIjoiNjFjZTI4ZDUtYWExNi00ZGRjLTg0MmYtZmNjMTU4ZDMxNWZhIiwiaXNzIjpudWxsLCJqdGkiOm51bGwsInBjdF9jbGFpbXMiOnsiYXVkIjpbIjYxY2UyOGQ1LWFhMTYtNGRkYy04NDJmLWZjYzE1OGQzMTVmYSJdLCJzdWIiOlsiZWIzMTQyMWUtMGEyZS00OTBmLWJiYWYtMDk3MWE0ZTliNzhhIl0sInVzZXJfbmFtZSI6WyJyaWNvbndheSJdLCJpc3MiOlsiaHR0cHM6Ly9lb2VwY2EtZGV2LmRlaW1vcy1zcGFjZS5jb20iXSwiZXhwIjpbIjE1OTM1MTU1NjQiXSwiaWF0IjpbIjE1OTM1MTE5NjQiXSwib3hPcGVuSURDb25uZWN0VmVyc2lvbiI6WyJvcGVuaWRjb25uZWN0LTEuMCJdfX0.d5qeaqLfl0oh9KigVrM_lT1hZMaOzQBFB7jjaKI3PjE"

In [ ]:
deploy_headers = {'Authorization': f'Bearer {token}',
                  'Content-Type': 'application/json',
                  'Accept': 'application/json', 
                  'Prefer': 'respond-async'} 

In [ ]:
deploy_payload = {'inputs': [{'id': 'applicationPackage',
                              'input': {'format': {'mimeType': 'application/xml'},
                                        'value': {'href': f'https://catalog.terradue.com/eoepca-services/search?uid={application_package_id}'}}}],
                  'outputs': [{'format': {'mimeType': 'string',
                                          'schema': 'string',
                                          'encoding': 'string'},
                               'id': 'deployResult',
                               'transmissionMode': 'value'}],
                  'mode': 'auto',
                  'response': 'raw'}

In [ ]:
r = requests.post(f'{endpoint}/wps3/processes',
                  json=deploy_payload,
                  headers=deploy_headers)

print(r.status_code, r.reason)

## /processes

In [ ]:
headers = {'Authorization': f'Bearer {token}',
           'Content-Type': 'application/json',
           'Accept': 'application/json'}

In [ ]:
r = requests.get(f'{endpoint}/wps3/processes',
                 headers=headers)

print(r.status_code, r.reason)

In [ ]:
response = r.json()

print(json.dumps(response, indent=2))

In [ ]:
for process in response['processes']:
    
    print('Process id: {}\n{} - {}\n'.format(process['id'], 
                                             process['title'], 
                                             process['abstract']))

## /processes/{id}

In [ ]:
process_id = 'vegetation_index_'

In [ ]:
r = requests.get(f'{endpoint}/wps3/processes/{process_id}',
                 headers=headers)

print(r.status_code, r.reason)

In [ ]:
response = r.json()

print(json.dumps(response, indent=2))

List the inputs

In [ ]:
inputs = response['process']['inputs']

In [ ]:
print(json.dumps(inputs, indent=2))

## Execute - /processes/{id}/jobs

In [ ]:
execute_request = {'inputs': [{'id': 'input_reference',
                               'input': {'dataType': {'name': 'string'},
                                         'value': 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808'}},
                              {'id': 'aoi',
                               'input': {'dataType': {'name': 'string'},
                                         'value': 'POLYGON((136.112726861895 -36.227897298303,137.333826362695 -36.2103069464338,137.305145407058 -35.2211228310596,136.099040812374 -35.2380875358202,136.112726861895 -36.227897298303))'}}],
                   'outputs': [{'format': {'mimeType': 'string',
                                           'schema': 'string',
                                           'encoding': 'string'},
                                'id': 'wf_output',
                                'transmissionMode': 'value'}],
                   'mode': 'async',
                   'response': 'raw'}

In [ ]:
execution_headers = {'Authorization': f'Bearer {token}',
                       'Content-Type': 'application/json',
                       'Accept': 'application/json', 
                       'Prefer': 'respond-async'}

In [ ]:
process_id = 'vegetation_index_'

In [ ]:
r = requests.post(f'{endpoint}/wps3/processes/{process_id}/jobs',
                  json=execute_request,
                  headers=execution_headers)

print(r.status_code, r.reason)

In [ ]:
job_location = r.headers['Location']

job_location

## Poll /processes/{id}/jobs/{job_id}

In [ ]:
r = requests.get(f'{endpoint}{job_location}',
                 headers=headers)

print(r.status_code, r.reason)

In [ ]:
response = r.json()

print(json.dumps(response, indent=2))

In [ ]:

while response['status'] == 'running':

    r = requests.get(f'{endpoint}{job_location}',
                 headers=headers)
    
    response = r.json()
    
    if response['status'] == 'failed': 
    
        print(response)
        
        break
        
    if response['status'] == 'successful':  

        print(response['links'][0]['href'])
        
        break
    
    else:
        
        print('Polling - {}'.format(response['status']))
        
        sleep(30)
        

## Results - /processes/{id}/jobs/{job_id}/result

In [ ]:
print(json.dumps(response, indent=2))

In [ ]:
r = requests.get(f'{endpoint}/{job_location}/result',
                 headers=headers)

print(r.status_code, r.reason)

In [ ]:
response = r.json()

print(json.dumps(response, indent=2))

In [ ]:
results = json.loads(response['outputs'][0]['value']['inlineValue'])

print(json.dumps(results, indent=2))

## Inspect the results

Use the STAC catalog endpoint in the "Exploiting the result.ipynb" Notebook 

In [ ]:
stac_catalog_endpoint = results['stac:catalog']['href']

stac_catalog_endpoint